# Job Scheduling and Slurm

<img src='../Pictures/Remote_Computing/Slurm_Logo.png' width="200" height="40" alt='Slurm Logo'>

The remote computers we use work differently than typical computers. Instead of running everything on the same processor and enviornment like your local computers, high performance computers have dedicated processors for each type of job. When we login to Alpine, you should notice a string before your cursor that looks like ``` @login ```. This tells us that are on a __login node__. A node is a collection of processor cores. So, what this means is that we are navigating and running our bash scripts on a cpu (cpu means processor) that is specifically intended for logging in users. However, our highly computationally demanding codes don't run on the login node. Instead, they run on __compute nodes__. These are completely separate nodes to which we need to submit our calculations.

This tutorial will teach you how to submit simple jobs to the compute nodes.

Let's start with a simple python script that will just print a list of numbers to a file. Copy the following code to a file called ```job_script.py``` in your bin directory.

In [ ]:
#! /usr/bin/envs python3 

target_num = float(1e8) #number to count up to
writelines = ''
for number in range(target_num):
    writelines += str(number) + '\n' #write one long string
    

with open('count_file.txt', 'w') as f: #open count_file.txt and write the writelines string to it
    f.write(writelines)

We'll now move into the __scratch__ directory. This directory is different than the directories we've been working in because it belongs to a different __file system__. File systems are different partitions of storage within the supercomputer. The scratch file system is meant to run calculations in due to its high transfer speeds and large amount of storage

### Important!
The scratch file system __will delete your files after about 2 months__.
This ensures that there is always room in scratch for all user. It also means that you need to back up your data to a different file system. We'll cover that later. For now, we can work in the scratch directory without fear of losing anything important.

```cd``` into the scratch directory by typing ```cd /scratch```

Then ```cd``` into your own scratch directory by typing ``` cd alpine/USER ```, but replace USER with your CU identikey.

The full path to my scratch directory is: ```/scratch/alpine/cote3804```

Next, create a directory called _tutorial_ with ```mkdir tutorial```. ```cd``` into the new directory and create a new file with ```touch submit.sh```

```submit.sh``` will be the name of our _submission script_. The submission script does a few important things:
- Enters your job in the queue
- Allocates compute node resources to your job
- Tells the compute node what program to run
- Sets up the compute node's environment

### Writing the submission script
Copy the following code into a file called ```submit.sh``` in your tutorial directory in the scratch directory.

```bash
#!/bin/bash

#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --time=00:10:00
#SBATCH --partition=amilan
#SBATCH --output=sample-%j.out

/home/$USER/bin/job_script.py
```

We'll go through the script line-by-line. First, we have the shebang ```#!/bin/bash``` to let the computer know it's a bash script.

```bash
#SBATCH --nodes=1
#SBATCH --ntasks=1
```
Although these lines look like comments, they actually convey information to the job scheduler. Whenever you want to specify job scheduling parameters in a submission script, they can be specified starting with ```#SBATCH``` followed by [a tag](https://slurm.schedmd.com/pdfs/summary.pdf). In this case, the ```--nodes=1``` tag specifies that the job will only need one node and the ```--ntasks=1``` specifies the job only needs 1 cpu.

```bash
#SBATCH --time=00:10:00
```
This line specifies that the maximum amount of time the job will need it 10 minutes. If the job is still running after 10 minutes, it will be terminated. Any jobs submitted with a max time below 30 minutes will be considered test jobs by Alpine and will take higher priority over other jobs in the queue.

```bash
#SBATCH --partition=amilan
```
This line specifies the partition. Partitions are essentially groupings of compute nodes. Sometimes they are used to designate graphical processing nodes from cpu nodes. All you need to know at the moment is that you will always be using the partition ```amilan```, which is the cpu compute partition on Alpine.

```bash
#SBATCH --output=sample-%j.out
```
This line formats the name of the files output by slurm. In this case I specify that I want the file to start with ```sample``` followed by the job number and ```.out```. We'll see when we run the submission script that a file with the correct naming convention is written to the submission directory.

```bash
/home/$USER/bin/job_script.py
```
This line actually specifies what is going to be done by the compute node. It points to the job_script.py file that we wrote earlier in our bin directory. This line tells the compute node to run this script.

### Submitting the job

Now that we've specified what we want the job scheduler and the compute node to do, we can submit the job. To do this, we'll use a special slurm command, ```sbatch```.

Type ```sbatch submit.sh``` and you should see this reponse with a different job ID

<img src="../Pictures/Remote_Computing/Job_Submitted.png" alt="Job Submitted">

Our job is now in the queue and we can check its progress with the alias ```qme```.
<img src="../Pictures/Remote_Computing/qme.png" alt="qme">

We see that the job is named ```submit.sh``` because we didn't specify a name in the submission script. The ```PD``` string refers to the fact that the job is pending, meaning it is waiting in the queue until there is a compute cpu ready to run the job.

### Summary

You've now learned the basics of job submission and monitoring on Alpine. For more job submission script examples and video explanations, check Alpine's [documentation](https://curc.readthedocs.io/en/latest/running-jobs/batch-jobs.html).

We will continue to use job submission throughout the rest of the tutorial, so make sure you understand the basics of how it works. This is an incredibly common topic on YouTube and there are plenty of excellent videos explaining it in different levels of depth.